# Programming Assignment: Social Network Database

## Welcome!

In this module you've been learning how LLMs can help with many common tasks when working with databases. In this assignment you'll have an opportunity to put those skills into practice. You will be working with a database designed to represent a simple social network. Here's the tasks you'll need to complete:

1. Write code that reads data from a provided CSV file into the database
2. Develop functions that query the database

### Submission and Grading

This notebook will be graded based on four functions that you write. You can find starter code for all four functions below. Unit tests are provided to help you test your work.

### Working with the LLM:

- **GPT-4o is available:** You can use whichever LLM you prefer on this assignment, but [GPT-4o has been made avaiable](https://www.coursera.org/learn/ai-powered-software-and-system-design/ungradedLab/rSPHu/gpt-4o-environment-for-assignment-to-use-alongside-the-programming-assignment) in the ungraded lab that follows this assignment
- **Provide Context to Your LLM:** The LLM will need the context of the problem you're working on and the code already available to you in order to help develop solutions.
- **Work Iteratively and Test as You Go:** Remember, the LLM's advice might not always be spot-on. It's up to you to decide what's useful, test the code it provides, and iterate as you work towards a solution.

### Necessary imports

In [ ]:
import unittests
import submission_checker

### Database Schema

The function below defines the schema for the database you will be using in this assignment. The database represents a social network made up of people who can be friends with one another and join clubs. You cannot edit this function but should understand how it works and the database schema it defines. **Read this function carefully and share it with your LLM** to make sure you understand the schema before moving on.

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
import numpy as np
import os
import pandas as pd
import json

# Creates the database for the social network

def create_database():
    Base = declarative_base()

    friendships = Table('friendships', Base.metadata,
                        Column('person_id', Integer, ForeignKey('people.id'), primary_key=True),
                        Column('friend_id', Integer, ForeignKey('people.id'), primary_key=True))

    club_members = Table('club_members', Base.metadata,
                         Column('person_id', Integer, ForeignKey('people.id'), primary_key=True),
                         Column('club_id', Integer, ForeignKey('clubs.id'), primary_key=True))

    class Person(Base):
        __tablename__ = 'people'
        id = Column(Integer, primary_key=True)
        name = Column(String)
        age = Column(Integer)
        gender = Column(String)
        location = Column(String)
        friends = relationship("Person",
                               secondary=friendships,
                               primaryjoin=id == friendships.c.person_id,
                               secondaryjoin=id == friendships.c.friend_id)
        clubs = relationship("Club", secondary=club_members, back_populates="members")

    class Club(Base):
        __tablename__ = 'clubs'
        id = Column(Integer, primary_key=True)
        description = Column(String)
        members = relationship("Person", secondary=club_members, back_populates="clubs")

    if os.path.exists("social_network.db"):
        os.remove("social_network.db")
    engine = create_engine(f'sqlite:///{"social_network.db"}', echo=False)
    Base.metadata.create_all(engine)

    Session = sessionmaker(bind=engine)
    session = Session()

    return session, Club, Person, friendships, club_members

### Exercise 1: Load the Database from a CSV

The database defined in the `create_database` function does not yet contain any data. You'll need to write the code for the `load_data_from_csv` function below to populate the database. To do that, you'll use the `members.csv` file which is available in this lab in the same directory as this notebook. The data is stored in a single table with one row for each person. The Friendships column contains the IDs of everyone the person in that row considers a friend. The Clubs column contains the names of each club the person in that row is a part of.

A few important notes about this exercise:

* In order for your function to be graded properly, do not change the function name or its parameters.
* You shouldn't need to import additional libraries, and this can sometimes cause errors with the grader. If you absolutely must, do so within the same cell as the `load_data_from_csv` function.


**HINTS**:

- **Remember** that you should avoid blindly pasting LLM code into your solution. If a code doesn't run, try looking at the exception and understand what might be going on, then you can pass more information to the LLM.
- **Remember** to also pass the `create_database()` function so the LLM can understand the database structure.
- **Remember** to pass the necessary information about the `members.csv` structure. You can see it by clicking on the `members.csv` file in the right panel. **Pass the correct information to the LLM regarding the columns,** **remember to mention that the Friends column is a list of integers corresponding to the friends' ID and the Clubs column is a list of strings corresponding to the clubs the member is in.**
- **Pass** the function `load_data_from_csv` and ask it to complete.
- **Remember** that the solution must be placed in the cell that is designated for it (the one below here). You may create new cells to test if you want to, but **remember** to paste your solution in the initial graded cell. Do **NOT** delete it, as it is tagged and the autograder will look for the specific tag, deleting it will make the autograder unable to find your solution.

In [ ]:
# The members.csv file contains 20 people. The code below will show you the first 5 rows to help you understand the format of the CSV file

pd.read_csv("members.csv", converters = {'Friendships': eval, "Clubs": eval}).head()

In [ ]:
# GRADED CELL 1 - Do NOT delete it, do NOT place your solution anywhere else. You can create new cells and work from there, but in the end add your solution in this cell.
# Function to load data from CSV into the database
def load_data_from_csv(session, Club, Person, friendships, club_members, csv_path="members.csv"):
    """
    Load data from a CSV file into the database, clearing existing data and creating new records for people, clubs, friendships, and club memberships.

    This function performs several key operations:
    1. Clears existing data from the Person, Club, friendships, and club_members tables.
    2. Reads data from a CSV file specified by `csv_path`, defaulting to "members.csv".
    3. For each row in the CSV, it creates or retrieves clubs mentioned, creates a person with the specified attributes, and associates them with the clubs.
    4. Establishes friendships based on the "Friendships" column in the CSV, which lists friend IDs for each person.
    5. Commits all changes to the database to ensure data integrity and consistency.

    Parameters:
    - session: The SQLAlchemy session object used for database transactions.
    - Club: The Club class model used to create or retrieve club records.
    - Person: The Person class model used to create person records.
    - friendships: The table/model representing friendships between people.
    - club_members: The table/model representing memberships of people in clubs.
    - csv_path (str, optional): The path to the CSV file containing the data to be loaded. Defaults to "members.csv".

    Returns:
    None. The function operates by side effects, modifying the database directly.

    Note:
    The function assumes the CSV file is formatted with specific columns: "ID", "Name", "Surname", "Age", "Gender", "Location", "Clubs", and "Friendships".
    "Clubs" is expected to be string representations of lists and "Friendships" is expected to be a list of IDs representing the member friends.
    """
    # Step 1: Clear existing data from all relevant tables
    session.query(Person).delete()
    session.query(Club).delete()
    session.query(friendships).delete()
    session.query(club_members).delete()

    session.commit()  # Commit the deletion of all existing records

    # Load the CSV data
    df = pd.read_csv("members.csv", converters = {'Friendships': eval, "Clubs": eval})
    ### START CODE HERE ###
    pass

In [ ]:
# The code below creates the database and reads in the data 
session, Club, Person, friendships, club_members = create_database()
load_data_from_csv(session, Club, Person, friendships, club_members, "members.csv")

# If your load_data_from_csv function is working correctly, then you should have read in data correctly into all four tables in the database.

print_amount = 3

# Print first 3 persons
print("=== All Persons ===")
people = session.query(Person).all()
for person in people[:print_amount ]:
    print(f"ID: {person.id}, Name: {person.name}, Age: {person.age}, Gender: {person.gender}, Location: {person.location}")

# Print first 3 clubs and their members
print("\n=== All Clubs and their Members ===")
clubs = session.query(Club).all()
for club in clubs[:print_amount ]:
    print(f"Club ID: {club.id}, Description: {club.description}, Members: {[member.name for member in club.members]}")

# Print friendships of first three persons
print("\n=== Friendships ===")
for person in people[:print_amount ]:
    friends = [friend.name for friend in person.friends]
    print(f"{person.name}'s Friends: {friends}")

Expected output:
```
=== All Persons ===
ID: 0, Name: John Rocha, Age: 57, Gender: Male, Location: 948 Christian Park Apt. 683 New Christopher, MN 06675
ID: 1, Name: William Ruiz, Age: 42, Gender: Female, Location: 09477 Paul Station Williamsmouth, NM 93899
ID: 2, Name: Jackie Mccullough, Age: 36, Gender: Non-binary, Location: 903 Keller River Suite 149 Davidside, IN 11476

=== All Clubs and their Members ===
Club ID: 1, Description: Fitness Club, Members: ['John Rocha', 'Amanda Norris', 'Michael Clark', 'Christina Murphy']
Club ID: 2, Description: Travel Club, Members: ['William Ruiz', 'Michael Powell', 'Becky Peterson', 'Nicholas Harrington', 'Luis Kim', 'Nathan Mendez']
Club ID: 3, Description: Art Club, Members: ['William Ruiz', 'Jackie Mccullough', 'Amanda Norris', 'Becky Peterson', 'Mark Allen', 'Brian Mays', 'Eric Dougherty', 'Nathan Mendez']

=== Friendships ===
John Rocha's Friends: ['Scott Boyd', 'Andrew Williams', 'Nicholas Harrington', 'Christina Murphy', 'Luis Kim', 'Mark Mcintyre']
William Ruiz's Friends: ['Michael Powell', 'Mark Allen', 'Nicholas Harrington', 'Brian Mays', 'Nathan Mendez']
Jackie Mccullough's Friends: ['Scott Boyd', 'Amanda Norris', 'Mark Allen', 'Nicholas Harrington']
```

In [ ]:
unittests.test_load_data_from_csv(load_data_from_csv)

### Exercise 2: Get a Club's Members

Assuming your data has been loaded correctly, you should now be able to query the data in your database. For this exercise, write a function called `get_club_members_by_description`. This function should accept a description of a club and a session, and return a list of all its members. **Ensure that this function returns a list containing the defined Person objects.** It must input only a **club description**.

In [ ]:
# GRADED CELL 2 - Do NOT delete it, do NOT place your solution anywhere else. You can create new cells and work from there, but in the end add your solution in this cell.
def get_club_members(session, club_description):
    """
    Returns a list of Person objects who are members of a club given the club's description.
    
    Parameters:
    - session: The SQLAlchemy session for database queries.
    - club_description (str): The description of the club for which members are to be retrieved.
    
    Returns:
    - List[Person]: A list of Person objects who are members of the specified club.
    """
    ### START CODE HERE ###
    pass

In [ ]:
# Example usage of the get_club_members function

# Assume the session and all models have been correctly set up and populated as per your initial code

# Fetching members of the "Hiking Club"
hiking_club_members = get_club_members(session, "Hiking Club")

# Printing out the names of all members of the Hiking Club
print("Members of the Hiking Club:")
for person in hiking_club_members:
    print(f"- {person.name}, Age: {person.age}, Location: {person.location}")

Expected output:
```
Members of the Hiking Club:
- Jackie Mccullough, Age: 36, Location: 903 Keller River Suite 149 Davidside, IN 11476
- Michael Powell, Age: 67, Location: 7528 Brenda Mills Jenniferstad, RI 38209
- Amanda Norris, Age: 27, Location: 8446 Thomas Crossing Suite 184 West Jacobshire, CA 41663
- Michael Clark, Age: 39, Location: 688 Sean River Apt. 585 Amandatown, AZ 76721
- Christina Murphy, Age: 19, Location: 276 Wolfe Springs Lake Eric, PA 61512
- Luis Kim, Age: 34, Location: 23367 Chase Heights Apt. 892 Calebchester, NC 59073
- Nathan Mendez, Age: 70, Location: 16039 Carey Views Apt. 776 Tylerfurt, DE 67034

```

In [ ]:
unittests.test_get_club_members(load_data_from_csv, get_club_members)

### Exercise 3: Get a Person's Friends

In this exercise, you are required to create a function named `get_friends_of_person`. This function should accept the name of a person and a session, return a list of all the people they consider to be friends. **Ensure that this function returns a list containing the defined Person objects.** The input must be only the **name of a person**.

In [ ]:
# GRADED CELL 3 - Do NOT delete it, do NOT place your solution anywhere else. You can create new cells and work from there, but in the end add your solution in this cell.
def get_friends_of_person(session, person_name):
    """
    Returns a list of Person objects who are friends with the specified person.
    
    Parameters:
    - session: The SQLAlchemy session object used to query the database.
    - person_name (str): The name of the person for whom to retrieve friends.
    
    Returns:
    - List[Person]: A list of Person objects who are friends with the specified person.
    """
    ### START CODE HERE ###
    pass

In [ ]:
# Example usage of the get_friends_of_person function

# Fetching friends of given name
name = "John Rocha"

john_friends = get_friends_of_person(session,name)

# Printing out the names of all friends of John Rocha
print(f"Friends of {name}:")
for friend in john_friends:
    print(f"- {friend.name}, Age: {friend.age}, Location: {friend.location}")

Expected output:
```
- Scott Boyd, Age: 47, Location: 545 Evelyn Shores Apt. 744 North Craigchester, CO 47369
- Andrew Williams, Age: 24, Location: USCGC Jacobs FPO AE 78539
- Nicholas Harrington, Age: 52, Location: 97681 Hernandez Villages Suite 344 Elizabethborough, UT 44717
- Christina Murphy, Age: 19, Location: 276 Wolfe Springs Lake Eric, PA 61512
- Luis Kim, Age: 34, Location: 23367 Chase Heights Apt. 892 Calebchester, NC 59073
- Mark Mcintyre, Age: 22, Location: 5635 Jennifer Freeway Suite 436 Newmanberg, IL 05315
```

In [ ]:
unittests.test_get_friends_of_person(load_data_from_csv, get_friends_of_person)

### Exercise 4: Get Everyone that Considers a Person a Friend

Now write a function called `get_persons_who_consider_them_friend`. This function should take two parameters: the name of an individual and a session. It will return a list of people who count this individual as a friend. It's important to remember that in our database, friendship isn't necessarily mutual. For example, Alice might consider Bob a friend, but Bob might not feel the same way about Alice. **Your function must return a list of Person objects for everyone who considers the input name their friend.** The input to this function should strictly be the **name of the person** you're inquiring about.

In [ ]:
# GRADED CELL 4 - Do NOT delete it, do NOT place your solution anywhere else. You can create new cells and work from there, but in the end add your solution in this cell.
def get_persons_who_consider_them_friend(session, person_name):
    """
    Returns a list of Person objects who consider the specified person as their friend,
    in a scenario where friendships are unidirectional.
    
    Parameters:
    - person_name (str): The name of the person to find who is considered as a friend by others.
    
    Returns:
    - List[Person]: A list of Person objects who consider the specified person as their friend.
    """
    ### START CODE HERE ###
    pass

In [ ]:
# Example usage of the get_persons_who_consider_them_friend function

# Fetching people who consider given name as their friend
name = 'John Rocha'

name_friend_of = get_persons_who_consider_them_friend(session, name)

# Printing out the names of all people who consider Alice as their friend
print(f"People who consider {name} as their friend:")
for person in name_friend_of:
    print(f"- {person.name}, Age: {person.age}, Location: {person.location}")

Expected output:
```
People who consider John Rocha as their friend:
- Scott Boyd, Age: 47, Location: 545 Evelyn Shores Apt. 744 North Craigchester, CO 47369
- Christina Murphy, Age: 19, Location: 276 Wolfe Springs Lake Eric, PA 61512
- Brian Mays, Age: 61, Location: 80208 Parker Glen Harrisland, PW 54882
- Nathan Mendez, Age: 70, Location: 16039 Carey Views Apt. 776 Tylerfurt, DE 67034
```

In [ ]:
unittests.test_get_persons_who_consider_them_friend(load_data_from_csv, get_persons_who_consider_them_friend)

## Preparing Your Submission for Grading

Your submission will be evaluated by an automated grading system, known as an autograder. This system automatically reviews your notebook and assigns a grade based on specific criteria. It's important to note that the autograder will only evaluate the cells marked for grading and will not consider the content of the entire notebook. Therefore, if you include any additional content (such as print statements) outside the functions in the graded cells, it might disrupt the autograder's process. This discrepancy could be why you might pass all the unit tests but still encounter issues with the autograder.

To avoid such problems, please execute the following cell before submitting. This step will check for consistency within the graded cells but will not evaluate the correctness of your solutions—that aspect is determined by the unit tests. If the consistency check uncovers any issues, you'll have the opportunity to review and adjust your code accordingly.

**Remember, this check is focused on ensuring the graded cells are properly formatted and does not assess the accuracy of your answers.**

In [ ]:
submission_checker.check_notebook()

Once you've written all four methods you're ready to submit. **Make sure you save your work before submitting** so that the grader will assess the most recent version of your assignment.

Congratulations on finishing this assignment!